In [3]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


## Notebook overview
- Get insight into tokenizer, tokens and doc lengths.
- Test different text truncation thresholds on the baseline.

#### Text truncation -- overview in tokenizer/doc lengths
- tokenize text using tokenizer of mistral, geitje and Llama.
- Check if mistral and geitje indeed have the same tokenizer.
- After getting the tokens, check distribution.
- Truncate text and test multiple thresholds on baseline

Results are saved in txtfiles_tokenizer.pkl, so that txtfiles.pkl is a back-up file, in case anything gets messed up

In [3]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")

KeyboardInterrupt: 

In [4]:
from transformers import AutoTokenizer

def get_tokens(model_name, df, save_to_path, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_tokens = []
    all_tokens_len = []
    for txt in all_texts:
        tokens = tokenizer.tokenize(txt)
        all_tokens.append(tokens)
        all_tokens_len.append(len(tokens))

    df[new_col_name] = all_tokens
    df[f"count_{new_col_name}"] = all_tokens_len
    df.to_pickle(save_to_path)
    return df

# subdf = df.iloc[0:2]
# # display(subdf)
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', subdf, f"{cf.output_path}/try_out_token_count.pkl", 'text', 'token_count_geitje')

def fraction_token(df, max_token, token_len_col):
    for col in token_len_col:
        print(f"{len(df.loc[df[col]>max_token])} out of {len(df)} ({round(len(df.loc[df[col]>max_token])/len(df)*100, 2)}%) docs exceed a token length of {max_token}")

    for col in token_len_col:
        print(df[col].describe())

    


    

##### Tokenize text

In [4]:
"""GEITje""" ## not necesarry -> since tokenizer is the same as mistral
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('Rijgersberg/GEITje-7B-chat-v2', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'GEITjeTokens')

"""Mistral"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'MistralTokens')

"""Llama"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'LlamaTokens')

'Llama'

##### Analyse token length of model tokenizers

In [2]:
import pandas as pd
tok = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# fraction_token(tok, 4096, ['count_MistralTokens', 'count_LlamaTokens'])

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>

#### Test text truncation on baseline

In [6]:
import itertools
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC


# load file with baseline function
import sys
sys.path.append('../scripts/') 
import baseline as bf

# load file with truncation function
from truncation import add_truncation_column

# variables for text truncation
DATAFRAME = tok
TEXT_COL = 'text'
TOKENS_COL = 'LlamaTokens'

# variables for baseline
BASELINE_FUNCTION = MultinomialNB()
MODEL_NAME = 'MultinomialNB'
TRAIN_SET = 'train' # must be dev or train
TEST_SET = 'test' # must be val or test
SPLIT_COLUMN = '4split' #must be either 2split or 4split. 2split = data split into train and test. 4split = data split into train, test, dev and val. 
LABEL_COLUMN = 'label'
PREDICTION_PATH = f"{cf.output_path}/predictions/baselineTruncationPredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/baselineTruncationOverview.pkl"
# PREDICTION_PATH = f"{cf.output_path}/predictions/tryoutBaselineTruncationPredictions.pkl"
# OVERVIEW_PATH = f"{cf.output_path}/overview/tryoutBaselineTruncationOverview.pkl"
TRUNC_COLUMN = 'trunc_txt'
threshold_combinations =[(100,0), (200,0), (500,0), (1000,0), (2000,0), (100,100),(200,200), (500,500), (1000,1000), (0,100), (0,200), (0,500), (0,1000), (0,2000)]
threshold_combinations =[(200,200), (500,500), (1000,1000), (0,100), (0,200), (0,500), (0,1000), (0,2000)]

# threshold_combinations = [(100,0)]

NameError: name 'tok' is not defined

In [12]:
# loop through all thresholds and save predictions
for thresholds in threshold_combinations:
    front_threshold = thresholds[0]
    back_threshold = thresholds[1]
    trunc = add_truncation_column(DATAFRAME, TEXT_COL, TOKENS_COL, front_threshold,back_threshold)
    bf.run_baseline(BASELINE_FUNCTION,MODEL_NAME, trunc, SPLIT_COLUMN, TRAIN_SET, TEST_SET, TRUNC_COLUMN, LABEL_COLUMN, PREDICTION_PATH, OVERVIEW_PATH)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                     precision    recall  f1-score   support

        Actualiteit       1.00      0.06      0.11       152
             Agenda       0.86      0.98      0.92       528
            Besluit       0.99      0.81      0.89       113
              Brief       0.80      0.79      0.79       206
          Factsheet       1.00      0.02      0.04        45
              Motie       0.84      0.97      0.90      1545
  Onderzoeksrapport       0.75      0.68      0.71       222
         Raadsadres       0.79      0.71      0.75       313
       Raadsnotulen       0.00      0.00      0.00        42
Schriftelijke Vraag       1.00      0.94      0.97       603
         Voordracht       0.94      0.99      0.97       395

           accuracy                           0.87      4164
          macro avg       0.81      0.63      0.64      4164
       weighted avg       0.87      0.87      0.84      4164



/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/e

                     precision    recall  f1-score   support

        Actualiteit       1.00      0.01      0.03       152
             Agenda       0.95      0.99      0.97       528
            Besluit       0.99      0.73      0.84       113
              Brief       1.00      0.08      0.15       206
          Factsheet       0.00      0.00      0.00        45
              Motie       0.70      1.00      0.82      1545
  Onderzoeksrapport       0.77      0.51      0.61       222
         Raadsadres       0.94      0.50      0.65       313
       Raadsnotulen       0.00      0.00      0.00        42
Schriftelijke Vraag       0.94      0.90      0.92       603
         Voordracht       0.97      0.98      0.98       395

           accuracy                           0.81      4164
          macro avg       0.75      0.52      0.54      4164
       weighted avg       0.83      0.81      0.77      4164



/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/e

...

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/e

                     precision    recall  f1-score   support

        Actualiteit       0.00      0.00      0.00       152
             Agenda       0.96      0.97      0.96       528
            Besluit       1.00      0.23      0.37       113
              Brief       1.00      0.01      0.02       206
          Factsheet       0.00      0.00      0.00        45
              Motie       0.55      1.00      0.71      1545
  Onderzoeksrapport       0.68      0.35      0.46       222
         Raadsadres       1.00      0.09      0.16       313
       Raadsnotulen       0.00      0.00      0.00        42
Schriftelijke Vraag       1.00      0.39      0.56       603
         Voordracht       0.98      0.99      0.99       395

           accuracy                           0.68      4164
          macro avg       0.65      0.37      0.39      4164
       weighted avg       0.75      0.68      0.61      4164



: 

: 

: 

In [18]:
import pandas as pd
PREDICTION_PATH = f"{cf.output_path}/predictions/baselineTruncationPredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/baselineTruncationOverview.pkl"

yeet = pd.read_pickle(OVERVIEW_PATH)
yeet = yeet.sort_values(by=['macro_avg_f1', 'accuracy'], ascending=False)
display(yeet)

bl = pd.read_pickle(f"{cf.output_path}/overview/baselineOverview.pkl")


,model,date,train_set,test_set,train_set_support,test_set_support,split_col,text_col,runtime,accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1,classification_report
0,LinearSVC,2024-04-24 17:00:16.579063+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront2000Back0,80.954455,0.960375,0.951155,0.906994,0.920938,precision recall f1-...
0,LinearSVC,2024-04-24 16:58:14.136467+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront500Back0,49.083472,0.960855,0.948753,0.903805,0.916650,precision recall f1-...
0,LinearSVC,2024-04-24 16:59:07.293244+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront1000Back0,64.067199,0.961335,0.950340,0.902041,0.916227,precision recall f1-...
0,LinearSVC,2024-04-24 17:07:29.160956+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront1000Back1000,146.854178,0.961095,0.948089,0.902357,0.915120,precision recall f1-...
0,LinearSVC,2024-04-24 17:05:07.923798+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront500Back500,134.567976,0.960375,0.946173,0.900545,0.911388,precision recall f1-...
0,LinearSVC,2024-04-24 17:03:08.622085+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront200Back200,113.316262,0.959174,0.952566,0.889015,0.902993,precision recall f1-...
0,LinearSVC,2024-04-24 17:22:30.597072+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront0Back2000,244.802182,0.949087,0.932871,0.887491,0.901655,precision recall f1-...
0,LinearSVC,2024-04-24 17:01:31.788353+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back100,93.574816,0.952209,0.945007,0.887995,0.901268,precision recall f1-...
0,LinearSVC,2024-04-24 16:57:41.418294+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront200Back0,28.803534,0.958213,0.948990,0.888563,0.899384,precision recall f1-...
0,LinearSVC,2024-04-24 16:57:27.306647+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back0,11.678588,0.952450,0.935202,0.883517,0.897058,precision recall f1-...


In [28]:

# ls = yeet.loc[yeet['model']=='LinearSVC']
ls = pd.concat([bl.loc[bl['model']=='LinearSVC'], yeet.loc[yeet['model']=='LinearSVC']]).sort_values(by=['macro_avg_f1', 'accuracy'], ascending=False)
ls = ls.loc[~ls['text_col'].isin(['TruncationLlamaTokensFront2000Back0', 'TruncationLlamaTokensFront1000Back0', 'TruncationLlamaTokensFront1000Back1000', 'TruncationLlamaTokensFront0Back2000',''])]

In [29]:
display(ls)

,model,date,train_set,test_set,train_set_support,test_set_support,split_col,text_col,runtime,accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1,classification_report
0,LinearSVC,2024-04-24 16:32:37.751511+02:00,train,test,15613,4164,4split,text,37.013646,0.961575,0.947063,0.907979,0.919503,precision recall f1-...
0,LinearSVC,2024-04-24 16:58:14.136467+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront500Back0,49.083472,0.960855,0.948753,0.903805,0.916650,precision recall f1-...
0,LinearSVC,2024-04-24 17:05:07.923798+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront500Back500,134.567976,0.960375,0.946173,0.900545,0.911388,precision recall f1-...
0,LinearSVC,2024-04-24 17:03:08.622085+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront200Back200,113.316262,0.959174,0.952566,0.889015,0.902993,precision recall f1-...
0,LinearSVC,2024-04-24 17:01:31.788353+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back100,93.574816,0.952209,0.945007,0.887995,0.901268,precision recall f1-...
0,LinearSVC,2024-04-24 16:57:41.418294+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront200Back0,28.803534,0.958213,0.948990,0.888563,0.899384,precision recall f1-...
0,LinearSVC,2024-04-24 16:57:27.306647+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back0,11.678588,0.952450,0.935202,0.883517,0.897058,precision recall f1-...
0,LinearSVC,2024-04-24 17:18:37.410520+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront0Back1000,228.073191,0.944284,0.920978,0.874937,0.889193,precision recall f1-...
0,LinearSVC,2024-04-24 17:15:31.851325+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront0Back500,181.190899,0.933718,0.915248,0.857465,0.875127,precision recall f1-...
0,LinearSVC,2024-04-24 17:12:31.622130+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront0Back200,176.178316,0.925072,0.918651,0.844159,0.865917,precision recall f1-...


In [32]:
nb = yeet.loc[yeet['model']=='MultinomialNB']
# nb = nb.loc[~nb['text_col'].isin(['TruncationLlamaTokensFront2000Back0', 'TruncationLlamaTokensFront1000Back0', 'TruncationLlamaTokensFront1000Back1000', 'TruncationLlamaTokensFront0Back2000',''])]
display(nb)



,model,date,train_set,test_set,train_set_support,test_set_support,split_col,text_col,runtime,accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1,classification_report
0,MultinomialNB,2024-04-29 14:00:56.847627+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back0,369.665684,0.866475,0.814319,0.632857,0.641090,precision recall f1-s...
0,MultinomialNB,2024-04-29 14:07:08.599145+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront200Back0,308.591510,0.808838,0.750644,0.518922,0.542854,precision recall f1-s...
0,MultinomialNB,2024-04-29 14:35:51.125123+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront100Back100,366.316200,0.776657,0.736425,0.470016,0.500990,precision recall f1-s...
0,MultinomialNB,2024-04-29 14:12:21.055348+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront500Back0,547.688317,0.630403,0.566161,0.324207,0.349426,precision recall f1-s...
0,MultinomialNB,2024-04-29 14:21:33.096035+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront1000Back0,444.322317,0.584774,0.561058,0.274303,0.282534,precision recall f1-s...
0,MultinomialNB,2024-04-29 14:29:04.029085+02:00,train,test,15613,4164,4split,TruncationLlamaTokensFront2000Back0,411.103931,0.583814,0.551679,0.271200,0.272104,precision recall f1-s...


### Finding best cut-off point